<a href="https://colab.research.google.com/github/cristian-gu/deep-learning-challenge/blob/main/nn_alphabetsoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts().sort_values(ascending=False)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_type = application_df["APPLICATION_TYPE"].value_counts().sort_values(ascending=False).index.tolist()
value_counts = application_df["APPLICATION_TYPE"].value_counts().sort_values(ascending=False).values.tolist()
other_df = pd.DataFrame({"app_type": app_type,
                        "value_counts": value_counts})

other_df = other_df[other_df["value_counts"] < 1000]
application_types_to_replace = other_df["app_type"].tolist()
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"App_Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3           27037
App_Other     2266
T4            1542
T6            1216
T5            1173
T19           1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_type = application_df["CLASSIFICATION"].value_counts().index.tolist()
class_val_counts = application_df["CLASSIFICATION"].value_counts().values.tolist()
class_df = pd.DataFrame({"class_type": class_type, "value_counts": class_val_counts})
nominal_class_df = class_df[class_df["value_counts"] > 1]
print(nominal_class_df)


   class_type  value_counts
0       C1000         17326
1       C2000          6074
2       C1200          4837
3       C3000          1918
4       C2100          1883
5       C7000           777
6       C1700           287
7       C4000           194
8       C5000           116
9       C1270           114
10      C2700           104
11      C2800            95
12      C7100            75
13      C1300            58
14      C1280            50
15      C1230            36
16      C1400            34
17      C7200            32
18      C2300            32
19      C1240            30
20      C8000            20
21      C7120            18
22      C1500            16
23      C1800            15
24      C6000            15
25      C1250            14
26      C8200            11
27      C1238            10
28      C1278            10
29      C1235             9
30      C1237             9
31      C7210             7
32      C2400             6
33      C1720             6
34      C4100       

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
other_class_df = class_df[class_df["value_counts"] < 1000]
classifications_to_replace = other_class_df["class_type"].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Class_Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000          17326
C2000           6074
C1200           4837
Class_Other     2261
C3000           1918
C2100           1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# reduce income features
application_df["INCOME_AMT"].value_counts()
income_type = application_df["INCOME_AMT"].value_counts().index.tolist()
income_vals = application_df["INCOME_AMT"].value_counts().values.tolist()

app_df = pd.DataFrame({"income_type": income_type, "income_vals": income_vals})
print(app_df)

     income_type  income_vals
0              0        24388
1    25000-99999         3747
2  100000-499999         3374
3          1M-5M          955
4         1-9999          728
5    10000-24999          543
6        10M-50M          240
7         5M-10M          185
8           50M+          139


In [10]:
# #Create cutoff values
# app_sig = app_df[app_df["income_vals"] < 500]["income_type"].tolist()

# for inc in app_sig:
#   application_df["INCOME_AMT"] = application_df["INCOME_AMT"].replace(inc, "income_other")
# application_df["INCOME_AMT"].value_counts()

In [11]:
application_df.head()
application_df["SPECIAL_CONSIDERATIONS"].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
classification_dummies = pd.get_dummies(application_df["CLASSIFICATION"])
use_dummies = pd.get_dummies(application_df["USE_CASE"])
org_dummies = pd.get_dummies(application_df["ORGANIZATION"])
income_dummies = pd.get_dummies(application_df["INCOME_AMT"])
aff_dummies = pd.get_dummies(application_df["AFFILIATION"])
app_dummies = pd.get_dummies(application_df["APPLICATION_TYPE"])

# Create the concatanation
application_df = pd.concat([application_df, classification_dummies, use_dummies, org_dummies, income_dummies, aff_dummies, app_dummies], axis=1)

# Drop unnecessary_columns
application_df = application_df.drop(columns = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT"])

# Change encoding for special_considerations column
def encode_considerations(considerations):
  if considerations == 'N':
    return 0
  else:
    return 1

application_df["SPECIAL_CONSIDERATIONS"] = application_df["SPECIAL_CONSIDERATIONS"].apply(encode_considerations)

application_df.head()

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,C1000,C1200,C2000,C2100,C3000,Class_Other,...,Independent,National,Other,Regional,App_Other,T19,T3,T4,T5,T6
0,1,0,5000,1,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,1,0,108590,1,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,1,0,5000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,6692,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,142590,1,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].ravel()
X = application_df.drop(columns = ["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [14]:
X.columns

Index(['STATUS', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'C1000', 'C1200',
       'C2000', 'C2100', 'C3000', 'Class_Other', 'CommunityServ', 'Heathcare',
       'Other', 'Preservation', 'ProductDev', 'Association', 'Co-operative',
       'Corporation', 'Trust', '0', '1-9999', '10000-24999', '100000-499999',
       '10M-50M', '1M-5M', '25000-99999', '50M+', '5M-10M', 'CompanySponsored',
       'Family/Parent', 'Independent', 'National', 'Other', 'Regional',
       'App_Other', 'T19', 'T3', 'T4', 'T5', 'T6'],
      dtype='object')

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 18
hidden_nodes_layer2 = 18
# hidden_nodes_layer3 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# # Third hidden layer
# nn.add(
#     tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu")
# )

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 18)                720       
                                                                 
 dense_1 (Dense)             (None, 18)                342       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 1081 (4.22 KB)
Trainable params: 1081 (4.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 7s 5ms/step - loss: 0.5913 - accuracy: 0.7011
Epoch 2/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5626 - accuracy: 0.7241
Epoch 3/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5583 - accuracy: 0.7284
Epoch 4/150
804/804 [==============================] - 3s 4ms/step - loss: 0.5564 - accuracy: 0.7297
Epoch 5/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7309
Epoch 6/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5544 - accuracy: 0.7309
Epoch 7/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5534 - accuracy: 0.7306
Epoch 8/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5527 - accuracy: 0.7324
Epoch 9/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5523 - accuracy: 0.7311
Epoch 10/150
804/804 [==============================] - 1s 2ms/step - loss: 0.5520 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5616 - accuracy: 0.7261 - 328ms/epoch - 1ms/step
Loss: 0.5616125464439392, Accuracy: 0.726064145565033


In [25]:
# Export our model to HDF5 file
nn.save("nn_model_Alphabet.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
